In [ ]:
import gzip
import json
import random
import time
from nltk import word_tokenize
from tqdm import tqdm
import contractions
import nltk
import random
import csv
import numpy as np
import tensorflow as tf

import gzip
import json
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer 
from sklearn.pipeline import Pipeline

from sklearn import preprocessing
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import LabelEncoder

from sklearn.metrics import f1_score
from sklearn.utils import shuffle
import re
from tensorflow.python.keras import layers, models

In [ ]:
#nltk.download('wordnet')
#nltk.download('omw-1.4')
#nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
random.seed(42)

In [ ]:
#here
def reading_in(path):
    thing=[]
    for line in gzip.open(path):
        review_data = json.loads(line)
        subthing = dict()
        for key in review_data:
            subthing[key]= review_data[key]
        thing.append(subthing)
    return thing

In [ ]:
train = reading_in('../data/classification/music_reviews_train.json.gz')
dev = reading_in('../data/classification/music_reviews_dev.json.gz') # do not use!
#test = reading_in('../data/classification/music_reviews_test_masked.json.gz') #do not use!
train = shuffle(train, random_state=42)

In [ ]:
train_x = [i['reviewText'] for i in train if "reviewText" in i.keys() and "sentiment" in i.keys()]
train_y =  [i['sentiment'] for i in train if "reviewText" in i.keys() and "sentiment" in i.keys()]

dev_x = [i['reviewText'] if "reviewText" in i.keys() else "" for i in dev]
dev_y = [i['sentiment'] for i in dev]

In [ ]:
###split into pos and neg
train_x_pos = []
train_x_neg = []
train_y_pos = []
train_y_neg = []
for i in range(len(train_x)):
    if train_y[i] == "positive":
        train_x_pos.append(train_x[i])
        train_y_pos.append(train_y[i])
    else:
        train_x_neg.append(train_x[i])
        train_y_neg.append(train_y[i])
train_x_neg.extend(train_x_pos[0:len(train_x_neg)])
train_y_neg.extend(train_y_pos[0:len(train_y_neg)])
balanced_x_train_s = train_x_neg
balanced_y_train_s = train_y_neg

balanced_x_train, balanced_y_train = shuffle(balanced_x_train_s, balanced_y_train_s, random_state= 42)
def tokenize_corpus(corpus):
    return [contractions.fix(s.lower()) for s in corpus]
balanced_x_train = tokenize_corpus(balanced_x_train)
dev_x = tokenize_corpus(dev_x)

### read in 3 synthetic

In [ ]:
def read_synth(path,deli):
    with open(path) as f:
        reader = csv.reader(f,delimiter = "\n")
        data = []
        for i in reader:
            i = contractions.fix(i[0].lower())
            data.append( re.sub(deli,' ',i))
           
    return data
paths = ["../data/synthetic/webscrape.csv","../data/synthetic/wordnet.csv","../data/synthetic/word2vec.csv"]
deli = [" ","\t","\t"]

In [ ]:
#balanced_x_train

In [ ]:
#synthetic["web"]

In [ ]:
synthetic = dict()
names= ["web","net","vec"]
for x,i in enumerate(paths):
    synthetic[names[x]] = read_synth(i,deli[x])

In [ ]:
def add_together(synt_dataset_X, samples):
    OG_X = balanced_x_train[:samples]
    OG_y = balanced_y_train[:samples]
    train_X = OG_X + synt_dataset_X[:samples]
    train_y = OG_y + OG_y
    train_X, train_y = shuffle(train_X, train_y, random_state= 42)
    return train_X, train_y

In [ ]:
Scrape10_X, Scrape10_y = add_together(synthetic['web'], 10000)
Net10_X, Net10_y = add_together(synthetic['net'], 10000)
Vec10_X, Vec10_y =add_together(synthetic['vec'],10000)
Scrape5_X, Scrape5_y = add_together(synthetic['web'], 5000)
Net5_X, Net5_y = add_together(synthetic['net'], 5000)
Vec5_X, Vec5_y = add_together(synthetic['vec'], 5000)

### list of datasets

In [ ]:
# Training X, training y,  description
train_list = [
    [balanced_x_train[:20000], balanced_y_train[:20000] ,'original data 20000', ],
    [balanced_x_train[:10000], balanced_y_train[:10000],'original data 10000'],
    [balanced_x_train[:5000], balanced_y_train[:5000], 'original data 5000'], 
    [Scrape10_X, Scrape10_y ,'WebScrape 10,000'],
    [Net10_X, Net10_y, 'WordNet 10,000'], 
    [ Vec10_X, Vec10_y,'Word2Vec 10,000' ],
    [Scrape5_X, Scrape5_y,'WebScrape 5,000'],
    [Net5_X, Net5_y, 'WordNet 5,000'], 
    [ Vec5_X, Vec5_y,'Word2Vec 5,000']
]

In [ ]:
#Tokenizer
def tokenize_corpus(sentence):
    return word_tokenize(contractions.fix(sentence))

In [ ]:
MNB = Pipeline([
    ('CountVectorizer',CountVectorizer(ngram_range=(1, 2), tokenizer = tokenize_corpus, lowercase = False )),
    ('Tfidf',TfidfTransformer()),
    ('clf', MultinomialNB(alpha=1, fit_prior=False))
])#best params given here


In [ ]:
result = []
cnt = 1

for run in train_list: 
    print(f'running run number {cnt}')
    MNB.fit(run[0], run[1])
    pred = MNB.predict(dev_x[:10000])
    result.append([run[2], f1_score(dev_y[:10000], pred, average='macro')])
    cnt += 1

running run number 1
running run number 2
running run number 3
running run number 4
running run number 5
running run number 6
running run number 7
running run number 8
running run number 9


In [ ]:
result

[['original data 20000', 0.8186222814613834],
 ['original data 10000', 0.8121945115351842],
 ['original data 5000', 0.7933088887447273],
 ['WebScrape 10,000', 0.8163322927363943],
 ['WordNet 10,000', 0.825087362561945],
 ['Word2Vec 10,000', 0.8236805607818262],
 ['WebScrape 5,000', 0.7966970867914357],
 ['WordNet 5,000', 0.8072615675482759],
 ['Word2Vec 5,000', 0.8059333957609268]]

# CNN
### Without embeddings
Best model (without the sigmoid activation) accuracy: 0.56 and error 0.67, was with 3 hidden layers with size [100,100,100], relu as activation, epochs 25, batch size of 50, filter size: 3, kernel size: 3, stride size: 3


In [ ]:
def pre_pros_y(data):
    encoder = LabelEncoder()
    encoder.fit(data)
    data = encoder.transform(data)
    return data

In [ ]:
def tokenize_corpus(corpus):
    return [contractions.fix(s) for s in corpus]

In [ ]:
def ready_CNN(data_X, data_y, dev_x, dev_y):
    #ADD TOKENIZER TO THE DEVELEPMENT DATA
    train_x_split = tokenize_corpus(data_X)

    vectorizer = TfidfVectorizer()
    tfid = vectorizer.fit(train_x_split)
    train_x = tfid.transform(train_x_split).toarray()
    train_x = np.reshape(train_x, (len(train_x), train_x.shape[1], 1))
    train_x = tf.convert_to_tensor(train_x)

    train_y = pre_pros_y(data_y)
    
    #devt_x = tokenize_corpus(dev_x[:10000]) is already tokenized
    devt_x = tfid.transform(devt_x)
    devt_x = dev_x.toarray()
    devt_x = np.reshape(devt_x, (len(devt_x), devt_x.shape[1], 1))
    devt_x = tf.convert_to_tensor(devt_x)

    devt_y = pre_pros_y(dev_y[:10000])
    return train_x, train_y, devt_x, devt_y



In [ ]:
act = 'relu'
model = models.Sequential()
model.add(layers.Conv1D(filters = 3, kernel_size = 3, strides = 3, padding = "same", activation='relu'))
model.add(layers.GlobalMaxPooling1D())
model.add(layers.Dense(100, activation=act))
model.add(layers.Dense(100, activation=act))
model.add(layers.Dense(100, activation=act))
model.add(layers.Dense(1, 'sigmoid'))

NameError: name 'models' is not defined

In [ ]:
for i in train_list[3:4]:
    train_x, train_y, devt_x, devt_y = ready_CNN(i[0], i[1],dev_x, dev_y )
print(train_x)

KernelInterrupted: Execution interrupted by the Jupyter kernel.

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adamax', metrics=['accuracy'])
history = model.fit(train_x, train_y, epochs=epoch, batch_size= batch, callbacks=[callback], verbose=0) #change the numbers
metrics = model.evaluate(dev_x,dev_y, verbose=0) #change the number


### With embeddings

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=b88dfe01-c7e1-473c-bcfd-798313fc6522' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>